### For Loops — Advanced Patterns (practical, not too insane)

These examples build on basic `for` loops with safe indexing, unpacking, `enumerate`, `zip`, `for`–`else`, sentinel iteration, dict iteration, controlled mutation, and a few handy recipes.

## 1) Loop unpacking & iteration over structured data

In [1]:
pairs = [("alice", 3), ("bob", 5), ("chris", 2)]
for name, count in pairs:
    print(f"{name=} {count=}")

# Nested unpacking
records = [(1, (10, 20)), (2, (30, 40))]
for rec_id, (x, y) in records:
    print(f"id={rec_id} → ({x}, {y})")

name='alice' count=3
name='bob' count=5
name='chris' count=2
id=1 → (10, 20)
id=2 → (30, 40)


## 2) Enumerate — indexes *and* values (with custom start)

In [2]:
data = [10, 20, 30]
for i, val in enumerate(data, start=1):  # start from 1
    print(f"row {i}: {val}")

row 1: 10
row 2: 20
row 3: 30


## 3) Zipping multiple iterables (parallel iteration)

In [3]:
names = ["A", "B", "C", "D"]
scores = [95, 88, 79]
weights = [0.5, 0.3, 0.2, 0.1]

# zip stops at the shortest iterable
for name, score, w in zip(names, scores, weights):
    print(f"{name}: score={score} weight={w}")

# If you need strict length matching, use zip with a check:
ok = len(names) == len(scores) == len(weights)
print("same length?", ok)

A: score=95 weight=0.5
B: score=88 weight=0.3
C: score=79 weight=0.2
same length? False


## 4) Dictionary iteration (keys, values, items)

In [4]:
cfg = {"host": "db.local", "port": 5432, "ssl": True}
for key in cfg:  # same as: for key in cfg.keys()
    print("key:", key)

for value in cfg.values():
    print("value:", value)

for key, value in cfg.items():
    print(f"{key} → {value}")

key: host
key: port
key: ssl
value: db.local
value: 5432
value: True
host → db.local
port → 5432
ssl → True


## 5) `for`–`else` (run `else` only if loop **didn't** `break`)

In [5]:
nums = [2, 4, 6, 9, 12]
for n in nums:
    if n % 2:  # found an odd
        print("Found odd:", n)
        break
else:  # runs only if no break happened
    print("All numbers were even")

Found odd: 9


## 6) `continue`, `break`, and guarded loops (skipping & early exit)

In [6]:
words = ["alpha", "", "beta", None, "gamma", "delta"]
for w in words:
    if not w:            # skip empty/None
        continue
    if w.startswith("g"):
        print("stop at:", w)
        break
    print("ok:", w)

ok: alpha
ok: beta
stop at: gamma


## 7) Sentinel iteration (no `while True`) using `iter(callable, sentinel)`

Loop until a callable returns the sentinel value. Great for chunked I/O or repeated API calls.

In [7]:
# Example: repeatedly read fixed-size chunks until b"" (EOF-like)
data = b"abcdefghij"  # pretend this is a stream
chunk_size = 3
pos = 0

def read_chunk():
    global pos
    if pos >= len(data):
        return b""  # sentinel
    chunk = data[pos:pos+chunk_size]
    pos += chunk_size
    return chunk

for chunk in iter(read_chunk, b""):
    print(chunk)

b'abc'
b'def'
b'ghi'
b'j'


## 8) Safe in-place modification while iterating

Avoid mutating a list **as you iterate over it directly**. Use index-based assignment or build a new list.

In [8]:
nums = [1, None, 3, None, 5]

# ✅ index-based update (safe)
for i, x in enumerate(nums):
    if x is None:
        nums[i] = 0
print(nums)

# ✅ create a new list (also safe & often clearer)
nums2 = [0 if x is None else x for x in nums]
print(nums2)

# ❌ removing/adding while iterating over the same list can skip elements

[1, 0, 3, 0, 5]
[1, 0, 3, 0, 5]


## 9) Looping over matrices: direct, with indexes, and transpose trick

In [9]:
m = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# Direct element iteration
for row in m:
    for x in row:
        pass  # do stuff

# With indexes (ragged rows supported)
for i, row in enumerate(m):
    for j, x in enumerate(row):
        if (i, j) in [(0, 0), (2, 2)]:
            print(f"diag @{i},{j} = {x}")

# Transpose with zip(*) (requires equal-length rows)
mt = list(zip(*m))  # columns as tuples
print(mt)

diag @0,0 = 1
diag @2,2 = 9
[(1, 4, 7), (2, 5, 8), (3, 6, 9)]


## 10) Accumulation patterns: any/all, running totals, grouping

In [10]:
nums = [3, 5, 8, 12, 15]

# any/all
has_even = any(n % 2 == 0 for n in nums)
all_pos  = all(n > 0 for n in nums)
print(has_even, all_pos)

# running total
total = 0
running = []
for n in nums:
    total += n
    running.append(total)
print(running)

# simple grouping (bucket by parity)
groups = {"even": [], "odd": []}
for n in nums:
    groups["even" if n % 2 == 0 else "odd"].append(n)
print(groups)

True True
[3, 8, 16, 28, 43]
{'even': [8, 12], 'odd': [3, 5, 15]}


## 11) Loop–`try/except` (robust processing of noisy data)

In [11]:
raw = ["10", "-3", "x", "5.5", None, "7"]
clean = []
for item in raw:
    try:
        clean.append(int(item))
    except (TypeError, ValueError):
        # log/skip/break as needed
        continue
print(clean)

[10, -3, 7]


## 12) Looping over files & lines safely (context manager)

In [12]:
# Demo content (normally you'd open an existing file)
path = "_demo.txt"
with open(path, "w", encoding="utf-8") as f:
    f.write("alpha\n\n beta\n gamma \n")

# Iterate lines lazily; strip whitespace; skip empty
clean_lines = []
with open(path, encoding="utf-8") as f:
    for line in f:
        s = line.strip()
        if not s:
            continue
        clean_lines.append(s)
clean_lines

['alpha', 'beta', 'gamma']

## 13) Batching / chunking indices with `range` (pairs, windows)

In [13]:
arr = list(range(1, 11))  # 1..10

# fixed-size non-overlapping batches (size=3)
size = 3
batches = [arr[i:i+size] for i in range(0, len(arr), size)]
print(batches)

# sliding window of size 3 (overlapping)
windows = [arr[i:i+3] for i in range(len(arr)-2)]
print(windows[:5])

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10]]
[[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7]]


## 14) Cartesian products (nested loops) via `itertools.product`

In [14]:
from itertools import product
rows = [0, 1, 2]
cols = [0, 1]
coords = list(product(rows, cols))  # (r,c) pairs
coords

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

## 15) Mini-exercises
1. Using `for`–`else`, search a list for a prime; print the prime or "no prime" if none found.
2. Read `_demo.txt` and build a dict `{line: len(line)}` for non-empty lines.
3. Given two equal-length lists of prices and quantities, compute total cost via `zip`.
4. Create 3×3 identity matrix using **one** nested loop and conditional.